Building A Chatbot

In these video. We will go over an example of how to design and LLM powered chatbots.This chatbot will be able to have a conservation and remeber previous interaction.

Note that this chatbot that we build will only use Language model to have a conservation. There are several other related concepts that you may be looking for :

. Conversational RAG: Enable a chatbot experience over an external source of data.
. Agents: Build a chatbot that can take actions.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## loading all env variables

groq_api_key = os.getenv("GROQ_API_KEY")


In [3]:

from langchain_groq import ChatGroq
model = ChatGroq(model = "gemma2-9b-it",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D3D8753700>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D3D87512A0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([
    HumanMessage(content="Hi,my name is naidu i am a chief AI enginner")
])

AIMessage(content="Hello Naidu, it's nice to meet you!\n\nThat's an impressive title! As a chief AI engineer, I'm sure you're involved in some exciting projects. \n\nWhat kind of work are you currently focused on?  I'd love to hear more about your role and the challenges and opportunities you're facing in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 23, 'total_tokens': 102, 'completion_time': 0.143636364, 'prompt_time': 0.002138888, 'queue_time': 0.06620019099999999, 'total_time': 0.145775252}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--3ad9392d-4b06-4b79-bac7-6cba1b17af50-0', usage_metadata={'input_tokens': 23, 'output_tokens': 79, 'total_tokens': 102})

In [5]:
from langchain_core.messages import AIMessage,HumanMessage

model.invoke(
    [
         HumanMessage(content="Hi,my name is naidu i am a chief AI enginner"),
         AIMessage(content="Hello Naidu, it's nice to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on?  \n\nI'm always eager to learn more about the innovative things people are doing with AI.\n"),
         HumanMessage(content="Hey what it's my name what I will do ?")

    ]
)

AIMessage(content="It seems like you're asking what you, as Naidu, the Chief AI Engineer, will do. That's a great question! \n\nAs a Chief AI Engineer, you'd likely be involved in a variety of exciting tasks, such as:\n\n* **Leading the development and implementation of AI solutions:** This could involve anything from building machine learning models to designing and deploying AI-powered applications.\n* **Researching and staying up-to-date on the latest AI advancements:** The field of AI is constantly evolving, so it's essential to be aware of the newest technologies and trends.\n* **Building and managing a team of AI engineers:** You'd be responsible for hiring, training, and mentoring a team of talented individuals.\n* **Collaborating with other departments:**  AI can have a wide range of applications across different industries. You'd likely work with teams from various departments to identify opportunities to leverage AI.\n* **Ensuring the ethical and responsible use of AI:**  As 

## Message History

We can use a Message History class to wrap our model and make its stateful.This will track of input and outputs of the model,and store them in a datastore.Future interactions will then load those messages and pass them into chain as a part of input .Let's see these.

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id:str)-> BaseChatMessageHistory:
 
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={
    "configurable": {"session_id":"chat1"}
}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(
        content = "Hi,my name is naidu i am a chief AI enginner")],
config=config
)

In [9]:
response=with_message_history.invoke(
    [HumanMessage(
        content = "Hi,my name is naidu i am a chief AI enginner")],
config=config
)

In [10]:
response.content

"Hello Naidu! It's great to meet you. \n\nBeing a Chief AI Engineer is a fascinating role. What are some of the most challenging and rewarding aspects of your work? \n\nI'm curious to hear about your experiences and the kind of projects you're involved in.  \n\n"

In [11]:
config2={
    "configurable": {"session_id":"chat2"}
}

In [12]:
response=with_message_history.invoke(
    [
        HumanMessage(content="my name is john")
    ],
    config=config2
)

In [13]:
response.content

"Hello John! 👋\n\nIt's nice to meet you. \n\nIs there anything I can help you with today? 😊\n"

In [14]:
### with diffeent session_id



response=with_message_history.invoke(
    [HumanMessage(
        content = "what's my name")],
config=config2
)
response.content

'Your name is John!  😊 \n'

## Prompt Templates
Prompt Template helps to turn raw user input into format of LLM can work with.In this case the raw input is a message,which we can passing to llm,First add system message with some custom instruction (but still take message as input),next we will add more input beside mesage.

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
pt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant i need your help for my question of your best ability"),
        MessagesPlaceholder(variable_name="messages"),
        
    ]
)

chain = pt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi, My name is krish")]})

AIMessage(content="Hello Krish! 👋 \n\nI'm happy to help. What can I do for you today? 😊  \n\nPlease ask away, and I'll do my best to provide a helpful answer!  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 32, 'total_tokens': 78, 'completion_time': 0.083636364, 'prompt_time': 0.002298138, 'queue_time': 0.083066268, 'total_time': 0.085934502}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--273bbd63-b261-440b-be08-d0c76864738f-0', usage_metadata={'input_tokens': 32, 'output_tokens': 46, 'total_tokens': 78})

In [17]:
with_message_history= RunnableWithMessageHistory(model,get_session_history)


In [18]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi,My name is krish")],
    config=config
)

In [19]:
response.content

"Hi Krish, it's nice to meet you! \n\nWhat can I do for you today? 😊  \n\n"

In [20]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

pt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your {language} ability."),
    MessagesPlaceholder(variable_name="messages")
])

chain = pt | model


In [21]:
from langchain_core.messages import HumanMessage

response = chain.invoke({
    "language": "Hindi",
    "messages": [HumanMessage(content="Hi, my name is Naidu")]
})

print(response.content)  # ✅ OR just: print(response)


नमस्ते नायडू!  मुझे खुशी है तुमसे मिलने में।  

(Namaste Naidu! It's nice to meet you.)

क्या मैं तुम्हें कुछ मदद कर सकता हूँ?

(Can I help you with something?) 




### Let's now wrap this more complicated chain in a Message Chat History class.This time ,it has multiple key in the input.we need to give a correct input to save the histyory chat.

In [22]:
with_message_history= RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [23]:
config={"configurable": {"session_id": "chat4"}}

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hi, my name is Naidu")],
        "language": "Hindi"
    },
    config=config
)
print(response)


content='नमस्ते नाइडू! \n\nमैं आपकी मदद करने के लिए यहाँ हूँ। आप क्या जानना चाहते हैं? \n(Namaste Naidu!\n\nI am here to help you. What would you like to know?) \n\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 32, 'total_tokens': 88, 'completion_time': 0.101818182, 'prompt_time': 0.002291098, 'queue_time': 0.065698704, 'total_time': 0.10410928}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--6fc8487b-cd56-4936-957c-60be436ab0ad-0' usage_metadata={'input_tokens': 32, 'output_tokens': 56, 'total_tokens': 88}


In [24]:
response =with_message_history.invoke({
    "messages":[HumanMessage(content="what's is my name")],"language":"Hindi",
},
config=config,
)
response.content

'आपका नाम नाइडू है। (Your name is Naidu.)  \n'

In [25]:
response =with_message_history.invoke({
    "messages":[HumanMessage(content="what's is my name")],"language":"Hindi",
},
config=config,
)
response.content

'आपका नाम नाइडू है।  😊 (Your name is Naidu.) \n'

### manage the conversation history
'''
one important concept to undersatnd when building chatbots is how to manage the conversation history.If left managed,the list of messages will grow unbounded potentially overflow the context window of LLM.Therefore it is important to add a step that limits the size of messages you passsing.

'''

"trim_message" helper to reduce how many messages we are sending to the model.The trimmer allow us  to specify how many token we want to keep,along with other parameters like if you want to keep output paramters like if we want to always keep the system message and wether to allow partial messages. 

In [31]:


from langchain_core.messages import trim_messages, SystemMessage, HumanMessage, AIMessage

# ✅ Messages list
messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="hi! I'm Bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what's 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

# ✅ Trim chat history based on token count
trimmed_messages = trim_messages(
    messages=messages,
    max_tokens=45,
    token_counter=model,          # model with get_num_tokens_from_messages()
    strategy="last",
    include_system=True,
    allow_partial=False,
    start_on="human"
)

for msg in trimmed_messages:
    print(msg.content)

you are a good assistant
I like vanilla ice cream
nice
what's 2+2
4
thanks
no problem
having fun?
yes!


In [35]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# Define a function to trim messages dynamically
def trim_input_messages(inputs):
    # Use the same trimming logic as before
    return trim_messages(
        messages=inputs["messages"],
        max_tokens=45,
        token_counter=model,
        strategy="last",
        include_system=True,
        allow_partial=False,
        start_on="human"
    )

chain = (
    RunnablePassthrough.assign(messages=trim_input_messages)
    | pt | model
)
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What Math Problem did I ask")],
        "language": "English"
    }
)

response.content

'You asked "What\'s 2+2". \n\n\n\nLet me know if you want to try another one! 😊\n'

In [36]:
## Lets wraps in a message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable":{"session_id":"chat5"}}



In [38]:
response =with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What Math Problem did I ask")],
        "language": "English"
    },
    config=config
)

response.content

'You asked "What\'s 2+2".  \n\n\n\nLet me know if you want to try another one! 😊 \n'

In [39]:
response =with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What's my name")],
        "language": "English"
    },
    config=config
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name. \n\nWould you like to tell me your name? 😊\n"